<a href="https://colab.research.google.com/github/rgukhui/cm4709/blob/main/Lab06_(PyTorch_version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CM4709 Computer Vision
#Lab 06 Object Detection using YOLO

##Aims
1. Use YOLO for object detection.

##Uploading Testing Images
We will need some testing images.
There are a few in Moodle. You can also add some of your own.
Upload them to a folder in your GoogleDrive. e.g. `cm4709/Lab06/data`.

##Mounting GoogleDrive

It is faster and easier uploading to GoogleDrive than to a Colab runtime.
The following standard code connects your GoogleDrive space to Colab.
After this, your image folder should be visible in your Colab runtime.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##Install YOLO v8

There are a few implementation of YOLO:
1. [Darknet](https://pjreddie.com/darknet/): This is the official release and is the fastest.
1. [Darkflow](https://github.com/thtrieu/darkflow): This is the Tensorflow version of Darknet.
1. [OpenCV](https://opencv-tutorial.readthedocs.io/en/latest/yolo/yolo.html): This is the OpenCV Implementation of YOLO. As YOLO is a CNN, you can also export a trained YOLO network to a ONNX file and run it in OpenCV C++. This is useful if you want to deploy a trained YOLO network to another platform.
1. [PyTorch](https://docs.ultralytics.com/): Ultralytics has released multiple versions of YOLO in Python. If you want the most up-to-date version, this will be the one to use.

In [ ]:
#
#install YOLO v8
#
!pip install ultralytics

import ultralytics
ultralytics.checks()

##Object Detection Using Command Line

The following command line loads the pretrained COCO `yolov8s.pt` weights and detects objects in an image.
Output will be saved in the folder `runs/detect/predict`.

In [ ]:
!yolo predict model=yolov8s.pt source='/content/gdrive/MyDrive/cm4709/Lab06/data/mumbai-traffic.jpg'

##Seeing the COCO Classes

The YOLO weights we loaded are trained with the [COCO dataset](https://cocodataset.org/). The COCO class names are in the result of detection.
The following code does object detecton on a testing image and print out the number of classes and the class labels.

The result documentation is [here](https://docs.ultralytics.com/modes/predict/#working-with-results). Note the followings:
1. We are feeding in 1 image in our example. Thus we are taking the first result.
1. The class names are in the `names` property of the result.

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO


#our 3 testing images
image1='/content/gdrive/MyDrive/cm4709/Lab06/data/mumbai-traffic.jpg'
image2='/content/gdrive/MyDrive/cm4709/Lab06/data/test.jpg'
image3='/content/gdrive/MyDrive/cm4709/Lab06/data/bluetit.png'
image=[image1,image2,image3]

model = YOLO('yolov8s.pt')  # load a pretrained YOLOv8s detection model
results=model(image)  # predict on an image
result=results[0]     #get 1st result
classNames=result.names #get class names

print('No. of classes: ',len(classNames))
print(classNames)

##Understanding the Detection Result

The Ultralytics library's detection function accepts multiple images in a list, and the results are returned as another list. Each result contains information of the class confidence score and bounding boxes. You can find documentation of the result [here](https://docs.ultralytics.com/modes/predict/).

The following code examines structure of the detection result.

In [ ]:
#Note: variable "results" is instantiated above

print('No. of results: ',len(results))

#go through all results
for result in results:
  boxes=result.boxes                    #get all boxes in this result
  print('  No. of boxes: ',len(boxes))
  for box in result.boxes:              #go through all boxes
    classIndex=box.cls                  #get class index
    confidence=box.conf                 #get confidence
    xywh=box.xywh                       #get bounding box
    print('    class index: ',classIndex)
    print('    confidence: ',confidence)
    print('    xywh: ',xywh)

##Processing the Result

The results are "tensors".
The following code extract the values and store them into 2 lists: 1 for the images, and 1 for the boxes detected.

In [ ]:
#minimum confidence required
confidenceThreshold=0.5

#lists to collect results
originalImages=[]
detectionResults=[]

for result in results:
  img=result.orig_img       #get original image
  originalImages.append(img)  #append image to list
  boxResults=[]
  for box in result.boxes:  #go through all boxes
    #
    #compute values from tensors
    #
    [classIndex]=box.cls.cpu().numpy()    #get class index of box
    classIndex=int(classIndex)            #convert to int
    label=classNames[classIndex]          #get class name as label
    [confidence]=box.conf.cpu().numpy()   #get box confidence
    label=label+' '+str(confidence)       #append confidence to label
    [[x,y,w,h]]=box.xywh.cpu().numpy()    #get box centre, width and height
    topx=int(x-w/2)       #compute box top left corner
    topy=int(y-h/2)
    bottomx=int(topx+w)   #compute box bottom right corner
    bottomy=int(topy+h)
    #
    #append box result to list
    #
    if confidence>=confidenceThreshold:
      boxResults.append((classIndex,confidence,label,(topx,topy,bottomx,bottomy)))
  detectionResults.append(boxResults)

#
#print out computed results
#
for boxResults in detectionResults:
  print('Image detection result:')
  boxIndex=0
  for boxResult in boxResults:
    (classIndex,confidence,label,(topx,topy,bottomx,bottomy))=boxResult
    print('  box no.: ',boxIndex)
    print('    class index: ',classIndex)
    print('    confidence: ',confidence)
    print('    label: ',label)
    print('    box: ',(topx,topy,bottomx,bottomy))
    boxIndex=boxIndex+1

##Visualising the Results

We can now visualise the bounding boxes.

In [ ]:
#font to be used in label
font = cv.FONT_HERSHEY_SIMPLEX

#generate random colours for the classes
colours = np.random.uniform(0, 255, size=(len(classNames), 3))

#
#go through all images
#
for index in range(0,len(originalImages)):
  image=originalImages[index]         #take out the image
  boxResults=detectionResults[index]  #get all box results of this image
  for boxResult in boxResults:        #go through all boxes in this result
    (classIndex,confidence,label,(topx,topy,bottomx,bottomy))=boxResult
    colour = colours[classIndex]                 #get colour
    cv.rectangle(image, (topx,topy), (bottomx,bottomy), colour, 2)  #draw bounding box
    cv.putText(image, label, (topx,topy -10), font, 0.5, colour)    #draw class label
  #
  #show image
  #
  plt.figure(figsize=(10,10))
  plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
